In [1]:
# Get ready
import lairningdecisions.server as server
import ray
import requests

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
backend_config = {'num_cpus': 0.5}
backend = server.start_backend_server(config=backend_config)
print(ray.available_resources())
print(ray.cluster_resources())

{'CPU': 1.0, 'object_store_memory': 63.0, 'node:178.18.249.39': 0.98, 'memory': 183.0}
{'object_store_memory': 63.0, 'node:178.18.249.39': 1.0, 'memory': 183.0, 'CPU': 6.0}


In [4]:
# Show Policies
# Todo: Refactor this code
backend.list_backends()


{'trainer1_policy3': BackendConfig(internal_metadata=BackendMetadata(accepts_batches=False, is_blocking=False, autoscaling_config=None), num_replicas=6, max_batch_size=None, batch_wait_timeout=0, max_concurrent_queries=1000000, user_config=None, experimental_graceful_shutdown_wait_loop_s=2.0, experimental_graceful_shutdown_timeout_s=20.0)}

In [5]:
# Show Endpoints and associated policies
backend.list_endpoints()

{'job_shop': {'route': '/job_shop',
  'methods': ['GET'],
  'traffic': {'trainer1_policy3': 1.0},
  'shadows': {}}}

In [5]:
# Show policies
server.show_policies()

,trainer_id,trainer_name,cloud_provider,model_name,sim_config_id,policy_id,run_id,time_start,simulations,duration,mean,std,deployed
0,1,job_shop,,job_shop,1,1,1,2021-08-05 23:06:13.162915,50,39.921585,606.3636,99.255508,False
1,1,job_shop,,job_shop,1,2,2,2021-08-05 23:06:53.173223,50,42.294294,556.6760,131.669631,False
2,1,job_shop,,job_shop,1,3,3,2021-08-05 23:07:35.553577,50,43.645915,596.9348,79.139637,False


In [20]:
#Deploy a policy, create an endpoint and associate the policiy with the endpoint
server.deploy_endpoint_policy(backend, trainer_id=1, policy_id=1, endpoint_name ="job_shop")
print(ray.available_resources())

# Policy 'trainer1_policy1' Deployed
{'object_store_memory': 63.0, 'node:178.18.249.39': 0.97, 'memory': 183.0, 'CPU': 3.5}


In [21]:
backend.update_backend_config('trainer1_policy1', config_options={'num_replicas':3})
print(ray.available_resources())

{'CPU': 2.5, 'node:178.18.249.39': 0.98, 'memory': 183.0, 'object_store_memory': 63.0}


In [22]:
# Test the endpoint by executing one simulation
job_shop_sim = server.get_simulator(trainer_id=1, policy_id=1)
observation = job_shop_sim.reset()
total_reward = 0
done = False
while not done:
    response = requests.get("http://localhost:8000/job_shop", json={"observation": observation})
    action = response.json()['action']
    observation, reward, done, _ = job_shop_sim.step(action)
    total_reward += reward

print(total_reward)


569.0400000000001


In [23]:
#Deploy a second policy
server.deploy_policy(backend, trainer_id=1, policy_id=3)
backend.update_backend_config('trainer1_policy3', config_options={'num_replicas':3})
print(ray.available_resources())

# Policy 'trainer1_policy3' Deployed
{'node:178.18.249.39': 0.98, 'memory': 183.0, 'CPU': 1.0, 'object_store_memory': 63.0}


In [24]:
#Split the traffic between the two policies
backend.set_traffic("job_shop", {'trainer1_policy1':0.5, 'trainer1_policy3':0.5})
backend.list_endpoints()

In [25]:
# Test the endpoint by executing one simulation
job_shop_sim = server.get_simulator(trainer_id=1, policy_id=1)
observation = job_shop_sim.reset()
total_reward = 0
done = False
while not done:
    response = requests.get("http://localhost:8000/job_shop", json={"observation": observation})
    action = response.json()['action']
    observation, reward, done, _ = job_shop_sim.step(action)
    total_reward += reward

print(total_reward)


610.5999999999998


In [6]:
#Remove the end point and it's associated policies
backend.delete_endpoint("job_shop")
server.undeploy_policy(backend, policy_name='trainer1_policy1')
server.undeploy_policy(backend, policy_name='trainer1_policy3')

In [30]:
#Deploy a policy, create an endpoint and associate the policiy with the endpoint
server.deploy_endpoint_policy(backend, trainer_id=1, policy_id=3, endpoint_name ="job_shop")
backend.update_backend_config('trainer1_policy3', config_options={'num_replicas':6})
print(ray.available_resources())

# Policy 'trainer1_policy3' Deployed
{'node:178.18.249.39': 0.97, 'object_store_memory': 63.0, 'memory': 183.0, 'CPU': 3.5}


In [32]:
# Get the KPIS for n Simulations
job_shop_sim = server.get_simulator(trainer_id=1, policy_id=1)
n = 20
revenue = 0
cost = 0
qtd = {'P1': 0, 'P2': 0, 'P3': 0}
duration = {'P1': 0, 'P2': 0, 'P3': 0}
for i in range(n):
    done = False
    observation = job_shop_sim.reset()
    while not done:
        response = requests.get("http://localhost:8000/job_shop", json={"observation": observation})
        action = response.json()['action']
        observation, _, done, kpis = job_shop_sim.step(action)
    revenue += kpis['Revenue']
    cost += kpis['Cost']
    avg_duration = dict()
    for p in qtd.keys():
        qtd[p] += kpis['Qtd'][p]
        duration[p] += kpis['Duration'][p]
avg_duration = dict()
for p in qtd.keys():
    avg_duration[p] = duration[p] / qtd[p]
print("# TOTAL # Avg Revenue={}; Avg Cost={}; Avg Duration={}".format(revenue / n, cost / n, avg_duration))

# TOTAL # Avg Revenue=971.9; Avg Cost=406.92700000000013; Avg Duration={'P1': 17.496132596685083, 'P2': 17.62213740458015, 'P3': 18.398780487804878}


In [17]:
#Remove the end point and it's associated policies
backend.delete_endpoint("job_shop")
server.undeploy_policy(backend, policy_name='trainer1_policy1')
server.undeploy_policy(backend, policy_name='trainer1_policy3')

In [33]:
# Show Policies
backend.list_backends()

{'trainer1_policy3': BackendConfig(internal_metadata=BackendMetadata(accepts_batches=False, is_blocking=False, autoscaling_config=None), num_replicas=6, max_batch_size=None, batch_wait_timeout=0, max_concurrent_queries=1000000, user_config=None, experimental_graceful_shutdown_wait_loop_s=2.0, experimental_graceful_shutdown_timeout_s=20.0)}

In [34]:
# Show Endpoints and associated policies
backend.list_endpoints()

{'job_shop': {'route': '/job_shop',
  'methods': ['GET'],
  'traffic': {'trainer1_policy3': 1.0},
  'shadows': {}}}